##Import data

In [2]:
# File location and type
file_location = "/FileStore/tables/WA_Fn_UseC__Telco_Customer_Churn-89c80.csv"
file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .option('nanValue',' ')\
  .option('nullValue',' ')\
  .load(file_location)

df.show(5)

+----------+------+-------------+-------+----------+------+------------+----------------+---------------+--------------+------------+----------------+-----------+-----------+---------------+--------------+----------------+--------------------+--------------+------------+-----+
customerID|gender|SeniorCitizen|Partner|Dependents|tenure|PhoneService| MultipleLines|InternetService|OnlineSecurity|OnlineBackup|DeviceProtection|TechSupport|StreamingTV|StreamingMovies| Contract|PaperlessBilling| PaymentMethod|MonthlyCharges|TotalCharges|Churn|
+----------+------+-------------+-------+----------+------+------------+----------------+---------------+--------------+------------+----------------+-----------+-----------+---------------+--------------+----------------+--------------------+--------------+------------+-----+
7590-VHVEG|Female| 0| Yes| No| 1| No|No phone service| DSL| No| Yes| No| No| No| No|Month-to-month| Yes| Electronic check| 29.85| 29.85| No|
5575-GNVDE| Male| 0| No| No| 34| Yes| No| DSL| Yes| No| Yes| No| No| No| One year| No| Mailed check| 56.95| 1889.5| No|
3668-QPYBK| Male| 0| No| No| 2| Yes| No| DSL| Yes| Yes| No| No| No| No|Month-to-month| Yes| Mailed check| 53.85| 108.15| Yes|
7795-CFOCW| Male| 0| No| No| 45| No|No phone service| DSL| Yes| No| Yes| Yes| No| No| One year| No|Bank transfer (au...| 42.3| 1840.75| No|
9237-HQITU|Female| 0| No| No| 2| Yes| No| Fiber optic| No| No| No| No| No| No|Month-to-month| Yes| Electronic check| 70.7| 151.65| Yes|
+----------+------+-------------+-------+----------+------+------------+----------------+---------------+--------------+------------+----------------+-----------+-----------+---------------+--------------+----------------+--------------------+--------------+------------+-----+
only showing top 5 rows

## Data Cleaning and data process

In [4]:
df.printSchema()

root
-- customerID: string (nullable = true)
-- gender: string (nullable = true)
-- SeniorCitizen: integer (nullable = true)
-- Partner: string (nullable = true)
-- Dependents: string (nullable = true)
-- tenure: integer (nullable = true)
-- PhoneService: string (nullable = true)
-- MultipleLines: string (nullable = true)
-- InternetService: string (nullable = true)
-- OnlineSecurity: string (nullable = true)
-- OnlineBackup: string (nullable = true)
-- DeviceProtection: string (nullable = true)
-- TechSupport: string (nullable = true)
-- StreamingTV: string (nullable = true)
-- StreamingMovies: string (nullable = true)
-- Contract: string (nullable = true)
-- PaperlessBilling: string (nullable = true)
-- PaymentMethod: string (nullable = true)
-- MonthlyCharges: double (nullable = true)
-- TotalCharges: double (nullable = true)
-- Churn: string (nullable = true)

Check NA/NULL value

In [6]:
from pyspark.sql.functions import isnan,when,count,col
df.select([count(when(isnan(c) | col(c).isNull(),c)).alias(c) for c in df.columns]).show()

+----------+------+-------------+-------+----------+------+------------+-------------+---------------+--------------+------------+----------------+-----------+-----------+---------------+--------+----------------+-------------+--------------+------------+-----+
customerID|gender|SeniorCitizen|Partner|Dependents|tenure|PhoneService|MultipleLines|InternetService|OnlineSecurity|OnlineBackup|DeviceProtection|TechSupport|StreamingTV|StreamingMovies|Contract|PaperlessBilling|PaymentMethod|MonthlyCharges|TotalCharges|Churn|
+----------+------+-------------+-------+----------+------+------------+-------------+---------------+--------------+------------+----------------+-----------+-----------+---------------+--------+----------------+-------------+--------------+------------+-----+
 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 11| 0|
+----------+------+-------------+-------+----------+------+------------+-------------+---------------+--------------+------------+----------------+-----------+-----------+---------------+--------+----------------+-------------+--------------+------------+-----+

In [7]:
display(df.select([count(when(isnan(c) | col(c).isNull(),c)).alias(c) for c in df.columns]))

customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,11,0


Only one cloumn monthly charge has 11 Null/NA values.

## Exploratory Data analysis

In [10]:
# Create a view or table

temp_table_name = "SQLtable"

df.createOrReplaceTempView(temp_table_name)

In [11]:
%sql

/* Query the created temp table in a SQL cell */

select * from SQLtable limit 5

customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.3,1840.75,No
9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.7,151.65,Yes


In [12]:
pd_df= df.toPandas()  #convert spark dataframe to Pandas dataframe,not good for very large dataset

In [13]:
import matplotlib.pyplot as plt
plt.clf()
plt.plot(pd_df['tenure'],pd_df['TotalCharges'],'.')
plt.xlabel('tenure')
plt.ylabel('total charges')
display()

In [14]:
#Our target predictor 'Churn' with 'yes' and 'no' values.
df.groupBy('Churn').count().show()

+-----+-----+
Churn|count|
+-----+-----+
 No| 5174|
 Yes| 1869|
+-----+-----+

In [15]:
#descript statistic data information
df.select('tenure','TotalCharges','MonthlyCharges').describe().show()


+-------+------------------+------------------+------------------+
summary| tenure| TotalCharges| MonthlyCharges|
+-------+------------------+------------------+------------------+
 count| 7043| 7032| 7043|
 mean| 32.37114865824223|2283.3004408418697| 64.76169246059922|
 stddev|24.559481023094442| 2266.771361883145|30.090047097678482|
 min| 0| 18.8| 18.25|
 max| 72| 8684.8| 118.75|
+-------+------------------+------------------+------------------+

In [16]:
%sql

select gender,churn,count(*) from SQLtable group by gender,churn

-- there is not much differece between male and female under two different churn.

gender,churn,count(1)
Male,No,2625
Male,Yes,930
Female,No,2549
Female,Yes,939


In [17]:
%sql

select SeniorCitizen,churn,count(*) from SQLtable group by SeniorCitizen,churn

-- for senior citizen,more people will stay on the same telecom business.

SeniorCitizen,churn,count(1)
1,No,666
0,No,4508
0,Yes,1393
1,Yes,476


In [18]:
%sql
select cast(tenure as int),churn,count(churn) from SQLtable group by tenure,churn order by cast(tenure as int)
-- With the incresing of the tenure year, the churn number decrease. There is a strong relationship between churn and tenure.

tenure,churn,count(churn)
0,No,11
1,No,233
1,Yes,380
2,No,115
2,Yes,123
3,No,106
3,Yes,94
4,No,93
4,Yes,83
5,No,69


In [19]:
display(df.stat.crosstab('SeniorCitizen','InternetService'))
# for internet service, most senior people choose fiber optic.There is some correlation but not causation.

SeniorCitizen_InternetService,DSL,Fiber optic,No
1,259,831,52
0,2162,2265,1474


In [20]:
df.stat.freqItems(['PhoneService','MultipleLines','InternetService','OnlineSecurity','OnlineBackup','DeviceProtection','TechSupport','StreamingTV','StreamingMovies'],0.75).collect()
# check the 70% most frquency results of above variables

Out[14]: [Row(PhoneService_freqItems=['Yes'], MultipleLines_freqItems=['Yes'], InternetService_freqItems=['Fiber optic'], OnlineSecurity_freqItems=['No'], OnlineBackup_freqItems=['Yes'], DeviceProtection_freqItems=['No'], TechSupport_freqItems=['No'], StreamingTV_freqItems=['Yes'], StreamingMovies_freqItems=['No'])]

In [21]:
%sql
select PaperlessBilling,churn,count(*) from SQLtable group by PaperlessBilling,churn
-- for paperlessbilling people are more likely to churn.

PaperlessBilling,churn,count(1)
Yes,Yes,1400
No,No,2403
Yes,No,2771
No,Yes,469


In [22]:
%sql
select PaymentMethod,churn,count(*) from SQLtable group by PaymentMethod,churn
-- Electronic check users are more likely to churn

PaymentMethod,churn,count(1)
Credit card (automatic),No,1290
Bank transfer (automatic),No,1286
Mailed check,Yes,308
Credit card (automatic),Yes,232
Electronic check,No,1294
Electronic check,Yes,1071
Bank transfer (automatic),Yes,258
Mailed check,No,1304


## Model building process

In [24]:
#devide the data set to training and testing date set by 70/30.
model_df = df
(train_data,test_data)= model_df.randomSplit([0.7,0.3],999)

print(str(train_data.count()))
print(str(test_data.count()))

4889
2154

In [25]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import OneHotEncoderEstimator,StringIndexer,VectorAssembler
# choose all categorical varibles
categColumn = ['gender','SeniorCitizen','Partner','Dependents','PhoneService','MultipleLines','InternetService','OnlineSecurity','OnlineBackup','DeviceProtection','TechSupport','StreamingTV','StreamingMovies','Contract','PaperlessBilling','PaymentMethod']

In [26]:
#Using pipeline to transform and estimate.
#1st pipeline transformer
stages = []

for catCol in categColumn:
  stringIndexer = StringIndexer(inputCol = catCol,outputCol = catCol +'Index')
  encoder = OneHotEncoderEstimator(inputCols = [stringIndexer.getOutputCol()],outputCols = [catCol +'catVec'])
  stages += [stringIndexer,encoder]

In [27]:
stages

Out[18]: [StringIndexer_011dea2877ae,
 OneHotEncoderEstimator_15a4bd5ebea9,
 StringIndexer_9a8eb5513948,
 OneHotEncoderEstimator_b19cdecf989d,
 StringIndexer_e30e0a570000,
 OneHotEncoderEstimator_ba142850a474,
 StringIndexer_103632e9704d,
 OneHotEncoderEstimator_6d81a959edad,
 StringIndexer_11de51a504e6,
 OneHotEncoderEstimator_64bac2d39b0c,
 StringIndexer_492ee54c871f,
 OneHotEncoderEstimator_fd6cbc1f2630,
 StringIndexer_3be8c29a49f2,
 OneHotEncoderEstimator_97927f7b5d29,
 StringIndexer_27fefbf59b07,
 OneHotEncoderEstimator_9da4348ca4c0,
 StringIndexer_59d8bf00412e,
 OneHotEncoderEstimator_b40dd28d91cc,
 StringIndexer_92617c75d10d,
 OneHotEncoderEstimator_626c8338287a,
 StringIndexer_411ab57fbbee,
 OneHotEncoderEstimator_f55445f2f2a3,
 StringIndexer_0bfd7267dd62,
 OneHotEncoderEstimator_a215b7c7dfb6,
 StringIndexer_d9eb32c7282e,
 OneHotEncoderEstimator_e5c53ed6861b,
 StringIndexer_8d6d45d683e9,
 OneHotEncoderEstimator_61ce20c9b088,
 StringIndexer_c4dde12a4217,
 OneHotEncoderEstimator_251317827152,
 StringIndexer_1cf7be90d23b,
 OneHotEncoderEstimator_dc7df7f687b9]

In [28]:
#2rd pipeline transformer
from pyspark.ml.feature import Imputer
imputer = Imputer(inputCols = ['TotalCharges'],outputCols = ['Out_TotalCharges'])
stages += [imputer]

In [29]:
temp2 = imputer.fit(train_data).transform(train_data)

In [30]:
temp2.show(5)

+----------+------+-------------+-------+----------+------+------------+-------------+---------------+--------------+------------+----------------+-----------+-----------+---------------+--------------+----------------+--------------------+--------------+------------+-----+----------------+
customerID|gender|SeniorCitizen|Partner|Dependents|tenure|PhoneService|MultipleLines|InternetService|OnlineSecurity|OnlineBackup|DeviceProtection|TechSupport|StreamingTV|StreamingMovies| Contract|PaperlessBilling| PaymentMethod|MonthlyCharges|TotalCharges|Churn|Out_TotalCharges|
+----------+------+-------------+-------+----------+------+------------+-------------+---------------+--------------+------------+----------------+-----------+-----------+---------------+--------------+----------------+--------------------+--------------+------------+-----+----------------+
0003-MKNFE| Male| 0| No| No| 9| Yes| Yes| DSL| No| No| No| No| No| Yes|Month-to-month| No| Mailed check| 59.9| 542.4| No| 542.4|
0004-TLHLJ| Male| 0| No| No| 4| Yes| No| Fiber optic| No| No| Yes| No| No| No|Month-to-month| Yes| Electronic check| 73.9| 280.85| Yes| 280.85|
0013-EXCHZ|Female| 1| Yes| No| 3| Yes| No| Fiber optic| No| No| No| Yes| Yes| No|Month-to-month| Yes| Mailed check| 83.9| 267.4| Yes| 267.4|
0013-MHZWF|Female| 0| No| Yes| 9| Yes| No| DSL| No| No| No| Yes| Yes| Yes|Month-to-month| Yes|Credit card (auto...| 69.4| 571.45| No| 571.45|
0013-SMEOE|Female| 1| Yes| No| 71| Yes| No| Fiber optic| Yes| Yes| Yes| Yes| Yes| Yes| Two year| Yes|Bank transfer (au...| 109.7| 7904.25| No| 7904.25|
+----------+------+-------------+-------+----------+------+------------+-------------+---------------+--------------+------------+----------------+-----------+-----------+---------------+--------------+----------------+--------------------+--------------+------------+-----+----------------+
only showing top 5 rows

In [31]:
stages

Out[22]: [StringIndexer_011dea2877ae,
 OneHotEncoderEstimator_15a4bd5ebea9,
 StringIndexer_9a8eb5513948,
 OneHotEncoderEstimator_b19cdecf989d,
 StringIndexer_e30e0a570000,
 OneHotEncoderEstimator_ba142850a474,
 StringIndexer_103632e9704d,
 OneHotEncoderEstimator_6d81a959edad,
 StringIndexer_11de51a504e6,
 OneHotEncoderEstimator_64bac2d39b0c,
 StringIndexer_492ee54c871f,
 OneHotEncoderEstimator_fd6cbc1f2630,
 StringIndexer_3be8c29a49f2,
 OneHotEncoderEstimator_97927f7b5d29,
 StringIndexer_27fefbf59b07,
 OneHotEncoderEstimator_9da4348ca4c0,
 StringIndexer_59d8bf00412e,
 OneHotEncoderEstimator_b40dd28d91cc,
 StringIndexer_92617c75d10d,
 OneHotEncoderEstimator_626c8338287a,
 StringIndexer_411ab57fbbee,
 OneHotEncoderEstimator_f55445f2f2a3,
 StringIndexer_0bfd7267dd62,
 OneHotEncoderEstimator_a215b7c7dfb6,
 StringIndexer_d9eb32c7282e,
 OneHotEncoderEstimator_e5c53ed6861b,
 StringIndexer_8d6d45d683e9,
 OneHotEncoderEstimator_61ce20c9b088,
 StringIndexer_c4dde12a4217,
 OneHotEncoderEstimator_251317827152,
 StringIndexer_1cf7be90d23b,
 OneHotEncoderEstimator_dc7df7f687b9,
 Imputer_b2e23ec197fa]

In [32]:
#3rd pipeline transformer
label_Idx = StringIndexer(inputCol = 'Churn',outputCol = 'label')
stages += [label_Idx]

In [33]:
#using label_Inx to transform training data
temp = label_Idx.fit(train_data).transform(train_data)

In [34]:
temp.show(5)
#we have label column after Churn which is show churn status in numeric variable

+----------+------+-------------+-------+----------+------+------------+-------------+---------------+--------------+------------+----------------+-----------+-----------+---------------+--------------+----------------+--------------------+--------------+------------+-----+-----+
customerID|gender|SeniorCitizen|Partner|Dependents|tenure|PhoneService|MultipleLines|InternetService|OnlineSecurity|OnlineBackup|DeviceProtection|TechSupport|StreamingTV|StreamingMovies| Contract|PaperlessBilling| PaymentMethod|MonthlyCharges|TotalCharges|Churn|label|
+----------+------+-------------+-------+----------+------+------------+-------------+---------------+--------------+------------+----------------+-----------+-----------+---------------+--------------+----------------+--------------------+--------------+------------+-----+-----+
0003-MKNFE| Male| 0| No| No| 9| Yes| Yes| DSL| No| No| No| No| No| Yes|Month-to-month| No| Mailed check| 59.9| 542.4| No| 0.0|
0004-TLHLJ| Male| 0| No| No| 4| Yes| No| Fiber optic| No| No| Yes| No| No| No|Month-to-month| Yes| Electronic check| 73.9| 280.85| Yes| 1.0|
0013-EXCHZ|Female| 1| Yes| No| 3| Yes| No| Fiber optic| No| No| No| Yes| Yes| No|Month-to-month| Yes| Mailed check| 83.9| 267.4| Yes| 1.0|
0013-MHZWF|Female| 0| No| Yes| 9| Yes| No| DSL| No| No| No| Yes| Yes| Yes|Month-to-month| Yes|Credit card (auto...| 69.4| 571.45| No| 0.0|
0013-SMEOE|Female| 1| Yes| No| 71| Yes| No| Fiber optic| Yes| Yes| Yes| Yes| Yes| Yes| Two year| Yes|Bank transfer (au...| 109.7| 7904.25| No| 0.0|
+----------+------+-------------+-------+----------+------+------------+-------------+---------------+--------------+------------+----------------+-----------+-----------+---------------+--------------+----------------+--------------------+--------------+------------+-----+-----+
only showing top 5 rows

In [35]:
df.stat.corr('TotalCharges','MonthlyCharges')
#there is some positive correlation bewteen TotalCharges and MonthlyCharges

Out[26]: 0.6511738315787813

In [36]:
%sql
select cast(tenure as int),churn,count(*) as churned from SQLtable where churn = 'Yes' group by tenure,churn order by cast(tenure as int)
-- there is some linear relationship between tenure and churn


tenure,churn,churned
1,Yes,380
2,Yes,123
3,Yes,94
4,Yes,83
5,Yes,64
6,Yes,40
7,Yes,51
8,Yes,42
9,Yes,46
10,Yes,45


In [37]:
#Add 4th pipeline transformation --- create 3 bins for tenure
from pyspark.ml.feature import QuantileDiscretizer
tenure_bin = QuantileDiscretizer(numBuckets =3,inputCol = 'tenure',outputCol = 'tenure_bin')
stages += [tenure_bin]

In [38]:
stages

Out[28]: [StringIndexer_011dea2877ae,
 OneHotEncoderEstimator_15a4bd5ebea9,
 StringIndexer_9a8eb5513948,
 OneHotEncoderEstimator_b19cdecf989d,
 StringIndexer_e30e0a570000,
 OneHotEncoderEstimator_ba142850a474,
 StringIndexer_103632e9704d,
 OneHotEncoderEstimator_6d81a959edad,
 StringIndexer_11de51a504e6,
 OneHotEncoderEstimator_64bac2d39b0c,
 StringIndexer_492ee54c871f,
 OneHotEncoderEstimator_fd6cbc1f2630,
 StringIndexer_3be8c29a49f2,
 OneHotEncoderEstimator_97927f7b5d29,
 StringIndexer_27fefbf59b07,
 OneHotEncoderEstimator_9da4348ca4c0,
 StringIndexer_59d8bf00412e,
 OneHotEncoderEstimator_b40dd28d91cc,
 StringIndexer_92617c75d10d,
 OneHotEncoderEstimator_626c8338287a,
 StringIndexer_411ab57fbbee,
 OneHotEncoderEstimator_f55445f2f2a3,
 StringIndexer_0bfd7267dd62,
 OneHotEncoderEstimator_a215b7c7dfb6,
 StringIndexer_d9eb32c7282e,
 OneHotEncoderEstimator_e5c53ed6861b,
 StringIndexer_8d6d45d683e9,
 OneHotEncoderEstimator_61ce20c9b088,
 StringIndexer_c4dde12a4217,
 OneHotEncoderEstimator_251317827152,
 StringIndexer_1cf7be90d23b,
 OneHotEncoderEstimator_dc7df7f687b9,
 Imputer_b2e23ec197fa,
 StringIndexer_34b35e70d059,
 QuantileDiscretizer_4395e71b8ec3]

In [39]:
# 5th pipeline transformation -- add numeric col
numericCols = ['tenure_bin','Out_TotalCharges','MonthlyCharges']
assembleInputs = assemblerInputs = [c+'catVec' for c in categColumn] + numericCols
assembler = VectorAssembler(inputCols = assembleInputs,outputCol = 'features')

stages += [assembler]

In [40]:
stages

Out[30]: [StringIndexer_011dea2877ae,
 OneHotEncoderEstimator_15a4bd5ebea9,
 StringIndexer_9a8eb5513948,
 OneHotEncoderEstimator_b19cdecf989d,
 StringIndexer_e30e0a570000,
 OneHotEncoderEstimator_ba142850a474,
 StringIndexer_103632e9704d,
 OneHotEncoderEstimator_6d81a959edad,
 StringIndexer_11de51a504e6,
 OneHotEncoderEstimator_64bac2d39b0c,
 StringIndexer_492ee54c871f,
 OneHotEncoderEstimator_fd6cbc1f2630,
 StringIndexer_3be8c29a49f2,
 OneHotEncoderEstimator_97927f7b5d29,
 StringIndexer_27fefbf59b07,
 OneHotEncoderEstimator_9da4348ca4c0,
 StringIndexer_59d8bf00412e,
 OneHotEncoderEstimator_b40dd28d91cc,
 StringIndexer_92617c75d10d,
 OneHotEncoderEstimator_626c8338287a,
 StringIndexer_411ab57fbbee,
 OneHotEncoderEstimator_f55445f2f2a3,
 StringIndexer_0bfd7267dd62,
 OneHotEncoderEstimator_a215b7c7dfb6,
 StringIndexer_d9eb32c7282e,
 OneHotEncoderEstimator_e5c53ed6861b,
 StringIndexer_8d6d45d683e9,
 OneHotEncoderEstimator_61ce20c9b088,
 StringIndexer_c4dde12a4217,
 OneHotEncoderEstimator_251317827152,
 StringIndexer_1cf7be90d23b,
 OneHotEncoderEstimator_dc7df7f687b9,
 Imputer_b2e23ec197fa,
 StringIndexer_34b35e70d059,
 QuantileDiscretizer_4395e71b8ec3,
 VectorAssembler_19c8db37bfd9]

In [41]:
#apply pipeline to training data
pipeline = Pipeline().setStages(stages)
pipelineModel = pipeline.fit(train_data)

In [42]:
#Transform the train data and test data
trainprepDF = pipelineModel.transform(train_data)
testprepDF = pipelineModel.transform(test_data)

In [43]:
trainprepDF.head(1)

Out[34]: [Row(customerID='0003-MKNFE', gender='Male', SeniorCitizen=0, Partner='No', Dependents='No', tenure=9, PhoneService='Yes', MultipleLines='Yes', InternetService='DSL', OnlineSecurity='No', OnlineBackup='No', DeviceProtection='No', TechSupport='No', StreamingTV='No', StreamingMovies='Yes', Contract='Month-to-month', PaperlessBilling='No', PaymentMethod='Mailed check', MonthlyCharges=59.9, TotalCharges=542.4, Churn='No', genderIndex=0.0, gendercatVec=SparseVector(1, {0: 1.0}), SeniorCitizenIndex=0.0, SeniorCitizencatVec=SparseVector(1, {0: 1.0}), PartnerIndex=0.0, PartnercatVec=SparseVector(1, {0: 1.0}), DependentsIndex=0.0, DependentscatVec=SparseVector(1, {0: 1.0}), PhoneServiceIndex=0.0, PhoneServicecatVec=SparseVector(1, {0: 1.0}), MultipleLinesIndex=1.0, MultipleLinescatVec=SparseVector(2, {1: 1.0}), InternetServiceIndex=1.0, InternetServicecatVec=SparseVector(2, {1: 1.0}), OnlineSecurityIndex=0.0, OnlineSecuritycatVec=SparseVector(2, {0: 1.0}), OnlineBackupIndex=0.0, OnlineBackupcatVec=SparseVector(2, {0: 1.0}), DeviceProtectionIndex=0.0, DeviceProtectioncatVec=SparseVector(2, {0: 1.0}), TechSupportIndex=0.0, TechSupportcatVec=SparseVector(2, {0: 1.0}), StreamingTVIndex=0.0, StreamingTVcatVec=SparseVector(2, {0: 1.0}), StreamingMoviesIndex=1.0, StreamingMoviescatVec=SparseVector(2, {1: 1.0}), ContractIndex=0.0, ContractcatVec=SparseVector(2, {0: 1.0}), PaperlessBillingIndex=1.0, PaperlessBillingcatVec=SparseVector(1, {}), PaymentMethodIndex=1.0, PaymentMethodcatVec=SparseVector(3, {1: 1.0}), Out_TotalCharges=542.4, label=0.0, tenure_bin=0.0, features=SparseVector(30, {0: 1.0, 1: 1.0, 2: 1.0, 3: 1.0, 4: 1.0, 6: 1.0, 8: 1.0, 9: 1.0, 11: 1.0, 13: 1.0, 15: 1.0, 17: 1.0, 20: 1.0, 21: 1.0, 25: 1.0, 28: 542.4, 29: 59.9}))]

In [44]:
trainprepDF.select('tenure_bin').show()

+----------+
tenure_bin|
+----------+
 0.0|
 0.0|
 0.0|
 0.0|
 2.0|
 0.0|
 2.0|
 2.0|
 0.0|
 2.0|
 2.0|
 2.0|
 1.0|
 0.0|
 1.0|
 1.0|
 0.0|
 2.0|
 2.0|
 1.0|
+----------+
only showing top 20 rows

## Fit model

In [46]:
from pyspark.ml.classification import LogisticRegression

#Create initial LogisticRegression model
lr = LogisticRegression(labelCol='label',featuresCol = 'features',maxIter = 10)

#Train model with Training data
lrModel = lr.fit(trainprepDF)

In [47]:
print('Coefficients: ' + str(lrModel.coefficients))
print('Intercept: ' + str(lrModel.intercept))

Coefficients: [-0.06284513268244098,-0.29369096968830755,0.0014864374861264982,0.061292876364840716,-0.5029857782401396,-0.2575494689739034,0.08475691341396997,0.5419750010243084,-0.35286881099327166,0.2669794959295581,-0.06414364001528323,0.1492305062199873,0.07508966515841715,0.1646761363924536,0.058069171857788725,0.28572901027024905,-0.08614583392526086,-0.010413602851078663,0.23852567474027736,-0.01907935624062497,0.24694842370492065,0.6093301029867657,-1.0436755447128194,0.24430377252393143,0.24962592277585688,-0.08320521103699956,-0.006553854982295144,-0.7495791335902192,-8.689977385906202e-05,0.005419378964940819]
Intercept: -1.2006946023722853

In [48]:
summary = lrModel.summary

In [49]:
accuracy = summary.accuracy
falsePositiveRate = summary.weightedFalsePositiveRate
turePositiveRate = summary.weightedTruePositiveRate
fMeasure = summary.weightedFMeasure()
precision = summary.weightedPrecision
recall = summary.weightedRecall
areaunderROC = summary.areaUnderROC
print('Accuracy: %s\nFPR: %s\nTPR: %s\nF-measure: %s\nPrecision: %s\nRecall: %s\nAreaUnderROC: %s' % (accuracy,falsePositiveRate,turePositiveRate,fMeasure,precision,recall,areaunderROC))

Accuracy: 0.8050726119860913
FPR: 0.37230178092457644
TPR: 0.8050726119860913
F-measure: 0.7977631838424428
Precision: 0.7955861706632565
Recall: 0.8050726119860913
AreaUnderROC: 0.8445502751280439

In [50]:
display(lrModel,trainprepDF,'ROC')

False Positive Rate,True Positive Rate,Threshold
0.0,0.0,0.8347161554445083
0.0,0.04,0.8347161554445083
0.0,0.08,0.767917251623131
0.0,0.12,0.7669676999142464
0.0,0.16,0.7352410974514162
0.015384615384615385,0.16,0.7337593855752097
0.03076923076923077,0.16,0.7094905721735388
0.03076923076923077,0.2,0.7089974406143809
0.046153846153846156,0.2,0.7077603306087087
0.06153846153846154,0.2,0.7016775170813181


In [51]:
display(lrModel,trainprepDF,'fittedVsResiduals')

fitted values,residuals
-0.8012651431623505,-0.3097549575466016
-2.8697155273418113,0.9463289013152797
-3.01138869714186,-0.046914013655980545
0.6391154686856171,0.3454465162665997
-3.3577735349944913,-0.033641530055226326
-2.6338847483020738,-0.06698923848058766
1.191267617308192,0.2330323000857536
-0.7107344691086965,0.6705634303170174
-4.884193924045399,-0.0075084166269284444
-0.32756754748944406,0.5811674062390477


## Evaluation

In [53]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
predictions = lrModel.transform(testprepDF)
evaluatorLR = BinaryClassificationEvaluator(rawPredictionCol = 'prediction')
area_under_curve = evaluatorLR.evaluate(predictions)

#default evaluation is areaunderROC
print('aresunderROC = %g' % area_under_curve)

evaluatorLR.getMetricName()

aresunderROC = 0.714644
Out[42]: 'areaUnderROC'

In [54]:
from pyspark.mllib.evaluation import BinaryClassificationMetrics

results = predictions.select(['prediction','label'])

# prepare score-label set
results_collect = results.collect()
results_list = [(float(i[0]),float(i[1])) for i in results_collect]
predictionAndLabels = sc.parallelize(results_list)

metrics = BinaryClassificationMetrics(predictionAndLabels)

#Area under precision-recall curve
print('Area under PR = %s' % metrics.areaUnderPR)

#Area under ROC curve
print('Area under ROC = %s' % metrics.areaUnderROC)

predictions.show(1)

Area under PR = 0.5674522568319178
Area under ROC = 0.714644095185967
+----------+------+-------------+-------+----------+------+------------+-------------+---------------+--------------+------------+----------------+-----------+-----------+---------------+--------+----------------+-------------+--------------+------------+-----+-----------+------------+------------------+-------------------+------------+-------------+---------------+----------------+-----------------+------------------+------------------+-------------------+--------------------+---------------------+-------------------+--------------------+-----------------+------------------+---------------------+----------------------+----------------+-----------------+----------------+-----------------+--------------------+---------------------+-------------+--------------+---------------------+----------------------+------------------+-------------------+----------------+-----+----------+--------------------+--------------------+--------------------+----------+
customerID|gender|SeniorCitizen|Partner|Dependents|tenure|PhoneService|MultipleLines|InternetService|OnlineSecurity|OnlineBackup|DeviceProtection|TechSupport|StreamingTV|StreamingMovies|Contract|PaperlessBilling|PaymentMethod|MonthlyCharges|TotalCharges|Churn|genderIndex|gendercatVec|SeniorCitizenIndex|SeniorCitizencatVec|PartnerIndex|PartnercatVec|DependentsIndex|DependentscatVec|PhoneServiceIndex|PhoneServicecatVec|MultipleLinesIndex|MultipleLinescatVec|InternetServiceIndex|InternetServicecatVec|OnlineSecurityIndex|OnlineSecuritycatVec|OnlineBackupIndex|OnlineBackupcatVec|DeviceProtectionIndex|DeviceProtectioncatVec|TechSupportIndex|TechSupportcatVec|StreamingTVIndex|StreamingTVcatVec|StreamingMoviesIndex|StreamingMoviescatVec|ContractIndex|ContractcatVec|PaperlessBillingIndex|PaperlessBillingcatVec|PaymentMethodIndex|PaymentMethodcatVec|Out_TotalCharges|label|tenure_bin| features| rawPrediction| probability|prediction|
+----------+------+-------------+-------+----------+------+------------+-------------+---------------+--------------+------------+----------------+-----------+-----------+---------------+--------+----------------+-------------+--------------+------------+-----+-----------+------------+------------------+-------------------+------------+-------------+---------------+----------------+-----------------+------------------+------------------+-------------------+--------------------+---------------------+-------------------+--------------------+-----------------+------------------+---------------------+----------------------+----------------+-----------------+----------------+-----------------+--------------------+---------------------+-------------+--------------+---------------------+----------------------+------------------+-------------------+----------------+-----+----------+--------------------+--------------------+--------------------+----------+
0002-ORFBO|Female| 0| Yes| Yes| 9| Yes| No| DSL| No| Yes| No| Yes| Yes| No|One year| Yes| Mailed check| 65.6| 593.3| No| 1.0| (1,[],[])| 0.0| (1,[0],[1.0])| 1.0| (1,[],[])| 1.0| (1,[],[])| 0.0| (1,[0],[1.0])| 0.0| (2,[0],[1.0])| 1.0| (2,[1],[1.0])| 0.0| (2,[0],[1.0])| 1.0| (2,[1],[1.0])| 0.0| (2,[0],[1.0])| 1.0| (2,[1],[1.0])| 1.0| (2,[1],[1.0])| 0.0| (2,[0],[1.0])| 2.0| (2,[],[])| 0.0| (1,[0],[1.0])| 1.0| (3,[1],[1.0])| 593.3| 0.0| 0.0|(30,[1,4,5,8,9,12...|[1.50269166245661...|[0.81797558498792...| 0.0|
+----------+------+-------------+-------+----------+------+------------+-------------+---------------+--------------+------------+----------------+-----------+-----------+---------------+--------+----------------+-------------+--------------+------------+-----+-----------+------------+------------------+-------------------+------------+-------------+---------------+----------------+-----------------+------------------+------------------+-------------------+--------------------+---------------------+-------------------+--------------------+---------------

In [55]:
test_data.groupBy('Churn').count().show()

+-----+-----+
Churn|count|
+-----+-----+
 No| 1566|
 Yes| 588|
+-----+-----+

In [56]:
#confusion matrix
count = predictions.count()
correct = results.filter(results.prediction == results.label).count()
wrong = results.filter(results.prediction != results.label).count()

tp = results.filter(results.prediction == 1.0).filter(results.prediction == results.label).count()
fp = results.filter(results.prediction == 1.0).filter(results.prediction != results.label).count()
fn = results.filter(results.prediction == 0.0).filter(results.prediction != results.label).count()
tn = results.filter(results.prediction == 0.0).filter(results.prediction == results.label).count()

accuracy = (tp+tn)/count

precision = tp/(tp+fp)

recall = tp/(tp+fn)

print('Correct: %s\nWrong: %s\ntp: %s\nfp: %s\nfn: %s\ntn: %s\naccuracy: %s\nprecision: %s\nrecall: %s'% (correct,wrong,tp,fp,fn,tn,accuracy,precision,recall))


Correct: 1716
Wrong: 438
tp: 314
fp: 164
fn: 274
tn: 1402
accuracy: 0.7966573816155988
precision: 0.6569037656903766
recall: 0.5340136054421769

In [57]:
# Cross validation
from pyspark.ml.tuning import ParamGridBuilder,CrossValidator
#create ParaGrid for cross validation
paraGrid = (ParamGridBuilder()
           .addGrid(lr.regParam,[0.01,0.5,2.0])
           .addGrid(lr.elasticNetParam,[0.0,0.5,1.0])
           .addGrid(lr.maxIter,[5,10,20])
           .build())

In [58]:
cv = CrossValidator(estimator = lr,estimatorParamMaps = paraGrid,evaluator = evaluatorLR,numFolds = 5)

#Run corss validation
cvModel = cv.fit(trainprepDF)

/databricks/spark/python/pyspark/ml/util.py:791: UserWarning: Can not find mlflow. To enable mlflow logging, install MLflow library from PyPi.
 warnings.warn(_MLflowInstrumentation._NO_MLFLOW_WARNING)

In [59]:
predictions = cvModel.bestModel.transform(testprepDF)

In [60]:
evaluatorLR.evaluate(predictions)

Out[50]: 0.713689498788021

In [61]:
#confusion matrix
count = predictions.count()
correct = results.filter(results.prediction == results.label).count()
wrong = results.filter(results.prediction != results.label).count()

tp = results.filter(results.prediction == 1.0).filter(results.prediction == results.label).count()
fp = results.filter(results.prediction == 1.0).filter(results.prediction != results.label).count()
fn = results.filter(results.prediction == 0.0).filter(results.prediction != results.label).count()
tn = results.filter(results.prediction == 0.0).filter(results.prediction == results.label).count()

accuracy = (tp+tn)/count

precision = tp/(tp+fp)

recall = tp/(tp+fn)

print('Correct: %s\nWrong: %s\ntp: %s\nfp: %s\nfn: %s\ntn: %s\naccuracy: %s\nprecision: %s\nrecall: %s'% (correct,wrong,tp,fp,fn,tn,accuracy,precision,recall))
#from the result,the cross validation did not make any significant improvement for our model.


Correct: 1716
Wrong: 438
tp: 314
fp: 164
fn: 274
tn: 1402
accuracy: 0.7966573816155988
precision: 0.6569037656903766
recall: 0.5340136054421769

In [62]:
cvModel.explainParams()

Out[52]: "estimator: estimator to be cross-validated (current: LogisticRegression_2add6dc52656)\nestimatorParamMaps: estimator param maps (current: [{Param(parent='LogisticRegression_2add6dc52656', name='regParam', doc='regularization parameter (>= 0).'): 0.01, Param(parent='LogisticRegression_2add6dc52656', name='elasticNetParam', doc='the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty.'): 0.0, Param(parent='LogisticRegression_2add6dc52656', name='maxIter', doc='max number of iterations (>= 0).'): 5}, {Param(parent='LogisticRegression_2add6dc52656', name='regParam', doc='regularization parameter (>= 0).'): 0.01, Param(parent='LogisticRegression_2add6dc52656', name='elasticNetParam', doc='the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty.'): 0.0, Param(parent='LogisticRegression_2add6dc52656', name='maxIter', doc='max number of iterations (>= 0).'): 10}, {Param(parent='LogisticRegression_2add6dc52656', name='regParam', doc='regularization parameter (>= 0).'): 0.01, Param(parent='LogisticRegression_2add6dc52656', name='elasticNetParam', doc='the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty.'): 0.0, Param(parent='LogisticRegression_2add6dc52656', name='maxIter', doc='max number of iterations (>= 0).'): 20}, {Param(parent='LogisticRegression_2add6dc52656', name='regParam', doc='regularization parameter (>= 0).'): 0.01, Param(parent='LogisticRegression_2add6dc52656', name='elasticNetParam', doc='the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty.'): 0.5, Param(parent='LogisticRegression_2add6dc52656', name='maxIter', doc='max number of iterations (>= 0).'): 5}, {Param(parent='LogisticRegression_2add6dc52656', name='regParam', doc='regularization parameter (>= 0).'): 0.01, Param(parent='LogisticRegression_2add6dc52656', name='elasticNetParam', doc='the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty.'): 0.5, Param(parent='LogisticRegression_2add6dc52656', name='maxIter', doc='max number of iterations (>= 0).'): 10}, {Param(parent='LogisticRegression_2add6dc52656', name='regParam', doc='regularization parameter (>= 0).'): 0.01, Param(parent='LogisticRegression_2add6dc52656', name='elasticNetParam', doc='the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty.'): 0.5, Param(parent='LogisticRegression_2add6dc52656', name='maxIter', doc='max number of iterations (>= 0).'): 20}, {Param(parent='LogisticRegression_2add6dc52656', name='regParam', doc='regularization parameter (>= 0).'): 0.01, Param(parent='LogisticRegression_2add6dc52656', name='elasticNetParam', doc='the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty.'): 1.0, Param(parent='LogisticRegression_2add6dc52656', name='maxIter', doc='max number of iterations (>= 0).'): 5}, {Param(parent='LogisticRegression_2add6dc52656', name='regParam', doc='regularization parameter (>= 0).'): 0.01, Param(parent='LogisticRegression_2add6dc52656', name='elasticNetParam', doc='the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty.'): 1.0, Param(parent='LogisticRegression_2add6dc52656', name='maxIter', doc='max number of iterations (>= 0).'): 10}, {Param(parent='LogisticRegression_2add6dc52656', name='regParam', doc='regularization parameter (>= 0).'): 0.01, Param(parent='LogisticRegression_2add6dc52656', name='elasticNetParam', doc='the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty.'): 1.0, Param(parent=

In [63]:
#Use random Forest to train the mode
from pyspark.ml.classification import RandomForestClassifier

rf = RandomForestClassifier(labelCol = 'label', featuresCol = 'features').setImpurity('gini').setMaxDepth(6).setNumTrees(50).setFeatureSubsetStrategy('auto').setSeed(999)

rfModel = rf.fit(trainprepDF)

In [64]:
predictions = rfModel.transform(testprepDF)

In [65]:
results = predictions.select(['prediction','label'])

count = predictions.count()
correct = results.filter(results.prediction == results.label).count()
wrong = results.filter(results.prediction != results.label).count()

tp = results.filter(results.prediction == 1.0).filter(results.prediction == results.label).count()
fp = results.filter(results.prediction == 1.0).filter(results.prediction != results.label).count()
fn = results.filter(results.prediction == 0.0).filter(results.prediction != results.label).count()
tn = results.filter(results.prediction == 0.0).filter(results.prediction == results.label).count()

accuracy = (tp+tn)/count

precision = tp/(tp+fp)

recall = tp/(tp+fn)

print('Correct: %s\nWrong: %s\ntp: %s\nfp: %s\nfn: %s\ntn: %s\naccuracy: %s\nprecision: %s\nrecall: %s'% (correct,wrong,tp,fp,fn,tn,accuracy,precision,recall))

Correct: 1712
Wrong: 442
tp: 260
fp: 114
fn: 328
tn: 1452
accuracy: 0.7948003714020427
precision: 0.6951871657754011
recall: 0.4421768707482993